In [1]:
import joblib
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc, \
  ConfusionMatrixDisplay, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score, matthews_corrcoef
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv(r'..\10_fold_cross_validation\train_10folds.csv')
data

,MaxPartialCharge,FpDensityMorgan2,BCUT2D_CHGLO,BCUT2D_MRHI,PEOE_VSA12,PEOE_VSA6,SMR_VSA3,SlogP_VSA3,SlogP_VSA8,EState_VSA6,NumHAcceptors,NumSaturatedCarbocycles,fr_bicyclic,TARGET,Kfold
0,0.335201,1.714286,-2.072068,5.975550,0.000000,12.132734,9.551078,0.000000,38.974594,43.638476,6,0,1,1.0,0
1,0.226791,2.000000,-2.362642,7.150190,5.948339,0.000000,9.967957,0.000000,10.440599,11.336786,6,0,0,1.0,1
2,0.211302,1.772727,-2.089721,7.912349,9.837253,29.297126,4.983979,9.837253,10.902925,24.265468,4,0,1,0.0,1
3,0.226898,1.869565,-2.357502,6.433493,5.948339,35.334614,15.284746,0.000000,11.126903,12.263211,5,0,0,0.0,0
4,0.158370,1.870968,-2.421374,7.991366,0.000000,23.362825,19.935914,6.420822,0.000000,18.460054,9,0,1,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,0.152613,2.000000,-2.415835,6.432407,0.000000,35.334614,9.967957,11.343745,11.257379,6.066367,6,0,0,1.0,3
2735,0.335203,1.700000,-2.001320,6.164147,0.000000,30.331835,4.983979,0.000000,10.902925,36.528679,2,0,1,0.0,6
2736,0.264178,1.833333,-2.414068,5.737009,5.948339,38.112943,9.551078,11.784535,11.126903,11.614772,7,0,0,1.0,8
2737,0.338995,1.815789,-2.065301,7.218862,5.907180,59.014740,24.544948,18.386966,43.634305,6.196844,9,0,2,0.0,5


In [3]:
def run(fold, data):
    # load the full training data with folds
    df = data
    # all columns are features except target and kfold columns
    features = [
        f for f in df.columns if f not in ("TARGET", "Kfold")
    ]
    # get training data using folds
    df_train = df[df.Kfold != fold].reset_index(drop=True)
    # get validation data using folds
    df_valid = df[df.Kfold == fold].reset_index(drop=True)
    # get training data
    X_train = df_train[features].values
    # get validation data
    X_valid = df_valid[features].values
    # initialize Logistic Regression model
    model = DecisionTreeClassifier()
    model.fit(X_train, df_train.TARGET.values)
    valid_preds = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(df_valid.TARGET.values, valid_preds)
    y_pred = model.predict(X_valid)
    y_true = df_valid.TARGET.values
    accuracy = accuracy_score(y_true,y_pred)
    precision_1 = precision_score(y_true,y_pred,pos_label=1)
    precision_0 = precision_score(y_true,y_pred,pos_label=0)
    recall_1 = recall_score(y_true,y_pred,pos_label=1)
    recall_0 = recall_score(y_true,y_pred,pos_label=0)
    f1score = f1_score(y_true,y_pred)
    kappa = cohen_kappa_score(y_true,y_pred)
    MCC = matthews_corrcoef(y_true,y_pred)
    print(f"Fold = {fold}, AUC = {auc}, Accuracy = {accuracy}, \
          Precision_1 = {precision_1}, Precision_0 = {precision_0}\
          Recall_1 = {recall_1}, Recall_0 = {recall_0}, F1Score = {f1score}, kappa = {kappa}, MCC = {MCC}")
    
    return auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model

In [4]:
aucs, accuracies, precisions_1, precisions_0, recalls_1, recalls_0, f1scores, kappas, MCCs = [], [], [], [], [], [], [], [], []

for fold_ in range(10):
    auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model = run(fold_, data)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions_1.append(precision_1)
    precisions_0.append(precision_0)
    recalls_1.append(recall_1)
    recalls_0.append(recall_0)
    f1scores.append(f1score)
    kappas.append(kappa)
    MCCs.append(MCC)
    #filename = 'SVM_' + str(fold_) + '.pkl'
    #joblib.dump(model, filename)
print("\n")
print(f"Mean Scores: AUC = {np.mean(np.array(aucs))}, \
      Accuracy = {np.mean(np.array(accuracies))}, \
      Precision_1 = {np.mean(np.array(precisions_1))}, Precision_0 = {np.mean(np.array(precisions_0))}\
      Recall_1 = {np.mean(np.array(recalls_1))}, Recall_0 = {np.mean(np.array(recalls_0))}\
      F1Score = {np.mean(np.array(f1scores))} \
      Kappa = {np.mean(np.array(kappas))} \
      MCC = {np.mean(np.array(MCCs))}")

Fold = 0, AUC = 0.9354068379055274, Accuracy = 0.9343065693430657,           Precision_1 = 0.9157894736842105, Precision_0 = 0.9441340782122905          Recall_1 = 0.8969072164948454, Recall_0 = 0.9548022598870056, F1Score = 0.9062499999999999, kappa = 0.8556966469658844, MCC = 0.85580665930699
Fold = 1, AUC = 0.9126041120624382, Accuracy = 0.9233576642335767,           Precision_1 = 0.9222222222222223, Precision_0 = 0.9239130434782609          Recall_1 = 0.8556701030927835, Recall_0 = 0.96045197740113, F1Score = 0.8877005347593583, kappa = 0.8296524364971283, MCC = 0.8309931849436243
Fold = 2, AUC = 0.913128312656532, Accuracy = 0.9087591240875912,           Precision_1 = 0.8913043478260869, Precision_0 = 0.9175824175824175          Recall_1 = 0.845360824742268, Recall_0 = 0.943502824858757, F1Score = 0.8677248677248677, kappa = 0.7981613530555719, MCC = 0.7988124722825258
Fold = 3, AUC = 0.9202341429320287, Accuracy = 0.916058394160584,           Precision_1 = 0.8557692307692307, Pre

In [5]:
fold_metrics = pd.DataFrame(columns=['Accuracy','AUC','Precision_1','Precision_0','Recall_1','Recall_0','F1score','Kappa','MCC'])
fold_metrics['Accuracy'] = np.array(accuracies)
fold_metrics['AUC'] = np.array(aucs)
fold_metrics['Precision_1'] = np.array(precisions_1)
fold_metrics['Precision_0'] = np.array(precisions_0)
fold_metrics['Recall_1'] = np.array(recalls_1)
fold_metrics['Recall_0'] = np.array(recalls_0)
fold_metrics['F1score'] = np.array(f1scores)
fold_metrics['Kappa'] = np.array(kappas)
fold_metrics['MCC'] = np.array(MCCs)
fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.934307,0.935407,0.915789,0.944134,0.896907,0.954802,0.906250,0.855697,0.855807
1,0.923358,0.912604,0.922222,0.923913,0.855670,0.960452,0.887701,0.829652,0.830993
2,0.908759,0.913128,0.891304,0.917582,0.845361,0.943503,0.867725,0.798161,0.798812
3,0.916058,0.920234,0.855769,0.952941,0.917526,0.915254,0.885572,0.819417,0.820657
4,0.927007,0.925243,0.873786,0.959064,0.927835,0.926554,0.900000,0.842610,0.843551
5,0.927007,0.926874,0.888889,0.948571,0.907216,0.937853,0.897959,0.841150,0.841256
6,0.927007,0.932029,0.905263,0.938547,0.886598,0.949153,0.895833,0.839663,0.839771
7,0.897810,0.896703,0.863158,0.916201,0.845361,0.926554,0.854167,0.775528,0.775628
8,0.927007,0.924772,0.887755,0.948864,0.906250,0.938202,0.896907,0.840419,0.840526
9,0.959707,0.959069,0.947368,0.966292,0.937500,0.971751,0.942408,0.911424,0.911453


In [6]:
fold_metrics.loc[10,:] = [np.mean(np.array(accuracies)), np.mean(np.array(aucs)), np.mean(np.array(precisions_1)),
                               np.mean(np.array(precisions_0)), np.mean(np.array(recalls_1)), np.mean(np.array(recalls_0)),
                            np.mean(np.array(f1scores)), np.mean(np.array(kappas)), np.mean(np.array(MCCs))]

fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.934307,0.935407,0.915789,0.944134,0.896907,0.954802,0.906250,0.855697,0.855807
1,0.923358,0.912604,0.922222,0.923913,0.855670,0.960452,0.887701,0.829652,0.830993
2,0.908759,0.913128,0.891304,0.917582,0.845361,0.943503,0.867725,0.798161,0.798812
3,0.916058,0.920234,0.855769,0.952941,0.917526,0.915254,0.885572,0.819417,0.820657
4,0.927007,0.925243,0.873786,0.959064,0.927835,0.926554,0.900000,0.842610,0.843551
5,0.927007,0.926874,0.888889,0.948571,0.907216,0.937853,0.897959,0.841150,0.841256
6,0.927007,0.932029,0.905263,0.938547,0.886598,0.949153,0.895833,0.839663,0.839771
7,0.897810,0.896703,0.863158,0.916201,0.845361,0.926554,0.854167,0.775528,0.775628
8,0.927007,0.924772,0.887755,0.948864,0.906250,0.938202,0.896907,0.840419,0.840526
9,0.959707,0.959069,0.947368,0.966292,0.937500,0.971751,0.942408,0.911424,0.911453


In [7]:
fold_metrics.loc[11,:] = [np.std(np.array(accuracies)), np.std(np.array(aucs)), np.std(np.array(precisions_1)),
                               np.std(np.array(precisions_0)), np.std(np.array(recalls_1)), np.std(np.array(recalls_0)),
                            np.std(np.array(f1scores)), np.std(np.array(kappas)), np.std(np.array(MCCs))]

fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.934307,0.935407,0.915789,0.944134,0.896907,0.954802,0.906250,0.855697,0.855807
1,0.923358,0.912604,0.922222,0.923913,0.855670,0.960452,0.887701,0.829652,0.830993
2,0.908759,0.913128,0.891304,0.917582,0.845361,0.943503,0.867725,0.798161,0.798812
3,0.916058,0.920234,0.855769,0.952941,0.917526,0.915254,0.885572,0.819417,0.820657
4,0.927007,0.925243,0.873786,0.959064,0.927835,0.926554,0.900000,0.842610,0.843551
5,0.927007,0.926874,0.888889,0.948571,0.907216,0.937853,0.897959,0.841150,0.841256
6,0.927007,0.932029,0.905263,0.938547,0.886598,0.949153,0.895833,0.839663,0.839771
7,0.897810,0.896703,0.863158,0.916201,0.845361,0.926554,0.854167,0.775528,0.775628
8,0.927007,0.924772,0.887755,0.948864,0.906250,0.938202,0.896907,0.840419,0.840526
9,0.959707,0.959069,0.947368,0.966292,0.937500,0.971751,0.942408,0.911424,0.911453


In [8]:
fold_metrics.index = ['Fold_0','Fold_1','Fold_2','Fold_3','Fold_4','Fold_5','Fold_6','Fold_7','Fold_8','Fold_9','Mean','Std']
fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
Fold_0,0.934307,0.935407,0.915789,0.944134,0.896907,0.954802,0.906250,0.855697,0.855807
Fold_1,0.923358,0.912604,0.922222,0.923913,0.855670,0.960452,0.887701,0.829652,0.830993
Fold_2,0.908759,0.913128,0.891304,0.917582,0.845361,0.943503,0.867725,0.798161,0.798812
Fold_3,0.916058,0.920234,0.855769,0.952941,0.917526,0.915254,0.885572,0.819417,0.820657
Fold_4,0.927007,0.925243,0.873786,0.959064,0.927835,0.926554,0.900000,0.842610,0.843551
Fold_5,0.927007,0.926874,0.888889,0.948571,0.907216,0.937853,0.897959,0.841150,0.841256
Fold_6,0.927007,0.932029,0.905263,0.938547,0.886598,0.949153,0.895833,0.839663,0.839771
Fold_7,0.897810,0.896703,0.863158,0.916201,0.845361,0.926554,0.854167,0.775528,0.775628
Fold_8,0.927007,0.924772,0.887755,0.948864,0.906250,0.938202,0.896907,0.840419,0.840526
Fold_9,0.959707,0.959069,0.947368,0.966292,0.937500,0.971751,0.942408,0.911424,0.911453


# Bayesian Optimization

In [13]:
from hyperopt import hp
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [17]:
class BayesianOPT:
    def __init__(self, data, space):
        self.space = space
        self.data = data
        self.trials = Trials()
    
    def _split_data(self, fold):
        df = self.data
        # all columns are features except target and kfold columns
        features = [
            f for f in df.columns if f not in ("TARGET", "Kfold")
        ]
        # get training data using folds
        df_train = df[df.Kfold != fold].reset_index(drop=True)
        # get validation data using folds
        df_valid = df[df.Kfold == fold].reset_index(drop=True)
        # get training data
        X_train = df_train[features].values
        # get validation data
        X_test = df_valid[features].values
        y_train = df_train.TARGET.values
        y_test = df_valid.TARGET.values
        
        return X_train, y_train, X_test, y_test
    
    def _objective(self,space):
        clf= DecisionTreeClassifier(**space)
        
        
        accuracies = []
        for fold_ in range(10):
            X_train, y_train, X_test, y_test = self._split_data(fold_)
            evaluation = [( X_train, y_train), ( X_test, y_test)]
        

            clf.fit(X_train, y_train)


            pred = clf.predict(X_test)
            accuracy = accuracy_score(y_test, pred>0.5)
            accuracies.append(accuracy)
        
        final_accuracy = np.mean(np.array(accuracies))
        #print ("SCORE:", final_accuracy)
        return {'loss': -final_accuracy, 'status': STATUS_OK }
    
    def search_hyperparameters(self):

        best_hyperparams = fmin(fn = self._objective,
                                space = self.space,
                                algo = tpe.suggest,
                                max_evals = 500,
                                trials = self.trials)
        return best_hyperparams

In [18]:
space = {
    'max_depth': hp.choice('max_depth', range(1, 10)),
    'criterion': hp.choice('criterion', ['gini', 'entropy']),
    'min_samples_split': hp.choice('min_samples_split', range(2, 10))
}

In [19]:
bayes_opt = BayesianOPT(data, space)
best_params = bayes_opt.search_hyperparameters()
print(best_params)

100%|█████████████████████████████████████████████| 500/500 [01:46<00:00,  4.67trial/s, best loss: -0.9050827517980803]
{'criterion': 0, 'max_depth': 7, 'min_samples_split': 1}


In [20]:
print(best_params)

{'criterion': 0, 'max_depth': 7, 'min_samples_split': 1}


In [30]:
best_params["min_samples_split"] = 2
best_params["criterion"] = "gini"

# Train the model with optimized hyperparameters

In [31]:
def run(fold, data, best_hyper_parameters):
    # load the full training data with folds
    df = data
    # all columns are features except target and kfold columns
    features = [
        f for f in df.columns if f not in ("TARGET", "Kfold")
    ]
    # get training data using folds
    df_train = df[df.Kfold != fold].reset_index(drop=True)
    # get validation data using folds
    df_valid = df[df.Kfold == fold].reset_index(drop=True)
    # get training data
    X_train = df_train[features].values
    # get validation data
    X_valid = df_valid[features].values
    # initialize Logistic Regression model
    model = DecisionTreeClassifier(**best_params)
    y_train = df_train.TARGET.values
    
    model.fit(X_train, y_train)
    valid_preds = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(df_valid.TARGET.values, valid_preds)
    y_pred = model.predict(X_valid)
    y_true = df_valid.TARGET.values
    accuracy = accuracy_score(y_true,y_pred)
    precision_1 = precision_score(y_true,y_pred,pos_label=1)
    precision_0 = precision_score(y_true,y_pred,pos_label=0)
    recall_1 = recall_score(y_true,y_pred,pos_label=1)
    recall_0 = recall_score(y_true,y_pred,pos_label=0)
    f1score = f1_score(y_true,y_pred)
    kappa = cohen_kappa_score(y_true,y_pred)
    MCC = matthews_corrcoef(y_true,y_pred)
    print(f"Fold = {fold}, AUC = {auc}, Accuracy = {accuracy}, \
          Precision_1 = {precision_1}, Precision_0 = {precision_0}\
          Recall_1 = {recall_1}, Recall_0 = {recall_0}, F1Score = {f1score}, kappa = {kappa}, MCC = {MCC}")
    
    return auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model

In [32]:
aucs, accuracies, precisions_1, precisions_0, recalls_1, recalls_0, f1scores, kappas, MCCs = [], [], [], [], [], [], [], [], []

for fold_ in range(10):
    auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model = run(fold_, data, best_params)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions_1.append(precision_1)
    precisions_0.append(precision_0)
    recalls_1.append(recall_1)
    recalls_0.append(recall_0)
    f1scores.append(f1score)
    kappas.append(kappa)
    MCCs.append(MCC)
    # filename = 'XGBoost_' + str(fold_) + '.pkl'
    # joblib.dump(model, filename)
print("\n")
print(f"Mean Scores: AUC = {np.mean(np.array(aucs))}, \
      Accuracy = {np.mean(np.array(accuracies))}, \
      Precision_1 = {np.mean(np.array(precisions_1))}, Precision_0 = {np.mean(np.array(precisions_0))}\
      Recall_1 = {np.mean(np.array(recalls_1))}, Recall_0 = {np.mean(np.array(recalls_0))}\
      F1Score = {np.mean(np.array(f1scores))} \
      Kappa = {np.mean(np.array(kappas))} \
      MCC = {np.mean(np.array(MCCs))}")

Fold = 0, AUC = 0.9463859281262741, Accuracy = 0.9087591240875912,           Precision_1 = 0.9390243902439024, Precision_0 = 0.8958333333333334          Recall_1 = 0.7938144329896907, Recall_0 = 0.9717514124293786, F1Score = 0.8603351955307262, kappa = 0.7932886716156679, MCC = 0.7994614180528328
Fold = 1, AUC = 0.8962374046246141, Accuracy = 0.8795620437956204,           Precision_1 = 0.9102564102564102, Precision_0 = 0.8673469387755102          Recall_1 = 0.7319587628865979, Recall_0 = 0.96045197740113, F1Score = 0.8114285714285714, kappa = 0.724480468035834, MCC = 0.7337717019301088
Fold = 2, AUC = 0.8922185334032268, Accuracy = 0.8576642335766423,           Precision_1 = 0.8452380952380952, Precision_0 = 0.8631578947368421          Recall_1 = 0.7319587628865979, Recall_0 = 0.9265536723163842, F1Score = 0.7845303867403314, kappa = 0.679079824614091, MCC = 0.6829989520097548
Fold = 3, AUC = 0.9264371832954744, Accuracy = 0.8759124087591241,           Precision_1 = 0.8387096774193549,